In [26]:
import boto3
import zipfile
import xml.etree.ElementTree as ET
import os

class Bucket:
    def __init__(
        self, 
        bucket_name:str, 
        access_key=None, 
        secret_key=None, 
        requester_pays='True', 
        prefix='',
        ):
        # Create an S3 client using the given access and secret keys
        self.s3_client = boto3.client('s3', aws_access_key_id=access_key, aws_secret_access_key=secret_key)
        
        self.bucket_name = bucket_name
        self.requester_pays = requester_pays
        self.prefix = prefix

    
    def list_objects(self):
        # List the objects in the bucket, with the given prefix and requester pays flag
        paginator = self.s3_client.get_paginator('list_objects_v2')
        page_iterator = paginator.paginate(Bucket=self.bucket_name, Prefix=self.prefix, RequestPayer='requester')
        for page in page_iterator:
            for object in page['Contents']:
                yield object
    
    def download_object(self, object_key, filename):
        # Download the given object from the bucket and save it to the given filename
        self.s3_client.download_file(
            Bucket=self.bucket_name, 
            Key=object_key, 
            Filename=filename, 
            ExtraArgs={'RequestPayer': self.requester_pays})
        
        # Return the manuscript title and the files
    def parse_manifest(self, zip_filename):
        # Extract the zip file and parse the manifest.xml file
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall()
        tree = ET.parse('manifest.xml')
        root = tree.getroot()
        
        # Return the manuscript title and the files in the content folder
        title = root.find('./title')
        files = []
        for file in root.findall('./file'):
            name = file.find('name').text
            size = file.find('size').text
            files.append((name, size))
        return title.text, files
    
    def cleanup(self, zip_filename):
        # Clean up the extracted files
        zip_ref.close()
        os.remove(zip_filename)
        os.remove('manifest.xml')

# Example usage


# Set the bucket name and the requester pays flag
bucket_name = 'biorxiv-src-monthly'
requester_pays = True

# Set the prefix to filter the objects by
prefix = ''

# Create a Bucket instance
bucket = Bucket(bucket_name=bucket_name)

objects = bucket.list_objects()


In [30]:
object = next(objects)
object

{'Key': 'Back_Content/Batch_01/006789a2-6c19-1014-a2c5-9e42b34c170a.meca',
 'LastModified': datetime.datetime(2019, 11, 19, 6, 50, 27, tzinfo=tzutc()),
 'ETag': '"2a9838726070714b44f14684cb230415"',
 'Size': 1191525,
 'StorageClass': 'STANDARD'}

In [ ]:
s3_client.download_file(
    Bucket='biorxiv-src-monthly', 
    Key='Current_Content/July_2022/ffee1f2e-6c3c-1014-a990-d896413aab59.meca', 
    Filename='temp.zip', 
    ExtraArgs={'RequestPayer': 'True'})

In [ ]:
bucket.download_object(object_key='Current_Content/July_2022/ffee1f2e-6c3c-1014-a990-d896413aab59.meca', filename='temp.zip')

In [ ]:

# Iterate over the objects in the bucket
for object in objects:
    # Download the object
    bucket.download_object(object['Key'], 'temp.zip')
    
    # Parse the manifest and print the title and file names and sizes
    title, files = bucket.parse_manifest('temp.zip')
    print(title)
    for name, size in files:
        print(f'{name}: {size} bytes')
    
    # Clean up
    bucket.cleanup('temp.zip')

    break



In [10]:
import requests
from bs4 import BeautifulSoup


url = 'https://www.biorxiv.org/content/10.1101/2022.12.16.520768v1?rss=1'
# Send an HTTP request to the URL of the webpage you want to access
response = requests.get(url)

# Parse the HTML content of the page
soup = BeautifulSoup(response.text, 'html.parser')


In [11]:
element = soup.find(name='div', class_='section abstract')

# Extract the text from the element
abstract_text = element.get_text()

In [12]:
element = soup.find('a', class_='article-dl-pdf-link')

In [13]:
pdf_path = element.get('href')

In [14]:
base_url = 'https://www.biorxiv.org'

In [15]:
pdf_full_url = f'{base_url}{pdf_path}'

In [7]:
pdf_full_url

'https://www.biorxiv.org/content/10.1101/2022.12.16.520768v1.full.pdf'

In [85]:
import requests
import tempfile

# Send an HTTP GET request to the URL
response = requests.get(url)

# Check that the response is successful
if response.status_code == 200:
    # Create a temporary file
    with tempfile.NamedTemporaryFile(mode='wb', delete=False) as f:
        # Write the contents of the response to the temporary file
        f.write(response.content)
        # Get the name of the temporary file
        temp_file_name = f.name
        print(f)

In [16]:
url = pdf_full_url

In [30]:
import requests
import tempfile
import PyPDF2 
from PIL import Image
import io

# Send an HTTP GET request to the URL
response = requests.get(url)

pil_images = []
texts = []
# Check that the response is successful
if response.status_code == 200:
    # Create a temporary file
    with tempfile.NamedTemporaryFile(mode='wb', delete=False) as f:
        # Write the contents of the response to the temporary file
        f.write(response.content)
        # Get the name of the temporary file
        temp_file_name = f.name

    # Open the PDF file in read-only mode
    with open(temp_file_name, 'rb') as f:
        # Create a PDF object
        pdf = PyPDF2.PdfFileReader(f)

        # Get the number of pages in the PDF
        num_pages = pdf.getNumPages()

        # Iterate over the pages
        for page_num in range(num_pages):
            # Get the page object
            page = pdf.getPage(page_num)
            # Extract the text from the page
            text = page.extractText()
            texts.append(text)
            

            # Get the resources dictionary
            resources = dict(page.getObject()['/Resources'])

            # Check if the resources dictionary has an /XObject key
            if '/XObject' in resources:
                # Get the XObjects from the resources dictionary
                xobjects = resources['/XObject']
                # Iterate over the XObjects
                for name, xobject in xobjects.items():
                    # Check if the XObject is an image
                    try:
                        if '/Subtype' in xobject and xobject['/Subtype'] == '/Image':
                        # Get the image object from the PDF file
                            image_obj = pdf.getObject(xobject).getObject()
                            # Get the image data from the image object
                            image_data = image_obj.getData()
                            # Create a PIL.Image object from the image data
                            image = Image.open(io.BytesIO(image_data))
                            # Do something with the image
                            pil_images.append(image)
                    except:
                        print(name)


                    



/Fm0
/Fm0
/Fm0
/Fm0
/Fm0
/Fm0
/Fm0
/Fm0
/Fm0
/Fm0
/Fm0
/Fm0
/Fm0
/Fm0
/Fm0
/Fm0


In [32]:
pil_images

[]

In [55]:
import fitz
from PIL import Image

# Send an HTTP GET request to the URL
response = requests.get(url)

pil_images = []
texts = []
# Check that the response is successful
assert response.status_code == 200

# Create a temporary file
with tempfile.NamedTemporaryFile(mode='wb', delete=False) as f:
    # Write the contents of the response to the temporary file
    f.write(response.content)
    # Get the name of the temporary file
    temp_file_name = f.name

pil_images = []
with fitz.open(temp_file_name) as pdf:
    # Iterate over the pages in the PDF
    for page in pdf:
        
        images = page.get_page_image_dict()
        for image_number, image in images.items():
            pil_image = image.get_pil_image()
            pil_images.append(pil_image)

AttributeError: 'Page' object has no attribute 'get_page_image_dict'

In [74]:
def recoverpix(doc, item):
    xref = item[0]  # xref of PDF image
    smask = item[1]  # xref of its /SMask

    # special case: /SMask or /Mask exists
    if smask > 0:
        pix0 = fitz.Pixmap(doc.extract_image(xref)["image"])
        if pix0.alpha:  # catch irregular situation
            pix0 = fitz.Pixmap(pix0, 0)  # remove alpha channel
        mask = fitz.Pixmap(doc.extract_image(smask)["image"])

        try:
            pix = fitz.Pixmap(pix0, mask)
        except:  # fallback to original base image in case of problems
            pix = fitz.Pixmap(doc.extract_image(xref)["image"])

        if pix0.n > 3:
            ext = "pam"
        else:
            ext = "png"

        return {  # create dictionary expected by caller
            "ext": ext,
            "colorspace": pix.colorspace.n,
            "image": pix.tobytes(ext),
        }

    # special case: /ColorSpace definition exists
    # to be sure, we convert these cases to RGB PNG images
    if "/ColorSpace" in doc.xref_object(xref, compressed=True):
        pix = fitz.Pixmap(doc, xref)
        pix = fitz.Pixmap(fitz.csRGB, pix)
        return {  # create dictionary expected by caller
            "ext": "png",
            "colorspace": 3,
            "image": pix.tobytes("png"),
        }
    return doc.extract_image(xref)
dimlimit = 0  # 100  # each image side must be greater than this
relsize = 0  # 0.05  # image : image size ratio must be larger than this (5%)
abssize = 0  # 2048  # absolute image size limit 2 KB: ignore if smaller

# Open the PDF file and get a Document object
doc = fitz.open(temp_file_name)

# Get the number of pages in the PDF
page_count = doc.page_count

# Initialize an empty list to store the images
images = []

imglist = []
xreflist = []
# Iterate over the pages in the PDF
for pno in range(page_count):
    il = doc.get_page_images(pno)
    imglist.extend([x[0] for x in il])
    for img in il:
        xref = img[0]
        if xref in xreflist:
            continue
        width = img[2]
        height = img[3]
        if min(width, height) <= dimlimit:
            continue
        image = recoverpix(doc, img)
        n = image["colorspace"]
        imgdata = image["image"]
        imglist.append(imgdata)
        xreflist.append(xref)

In [ ]:
Image.open(BytesIO(doc.extract_image(2494)['image']))

In [75]:
xreflist

[197,
 204,
 209,
 210,
 252,
 269,
 271,
 654,
 655,
 656,
 657,
 658,
 659,
 660,
 1058,
 1062,
 1066,
 1222,
 1106,
 1107,
 1113,
 1114,
 1159,
 1161,
 1286,
 1290,
 1726,
 1724,
 1929,
 1927,
 1994,
 1996,
 1998,
 2000,
 1992,
 2056,
 2055,
 2054,
 2053,
 2068,
 2067,
 2066,
 2065,
 2080,
 2079,
 2078,
 2077,
 2092,
 2091,
 2090,
 2089,
 2159,
 2158,
 2157,
 2156,
 2171,
 2170,
 2169,
 2168,
 2183,
 2182,
 2181,
 2180,
 2195,
 2194,
 2193,
 2192,
 2264,
 2265,
 2271,
 2272,
 2295,
 2296,
 2341,
 2343,
 2345,
 2347,
 2349,
 2351,
 2353,
 2355,
 2357,
 2359,
 2361,
 2363,
 2365,
 2335,
 2337,
 2339,
 2392,
 2394,
 2396,
 2398,
 2400,
 2402,
 2404,
 2406,
 2408,
 2410,
 2412,
 2414,
 2416,
 2386,
 2388,
 2390,
 2439,
 2441,
 2443,
 2445,
 2447,
 2449,
 2451,
 2453,
 2455,
 2433,
 2435,
 2437,
 2482,
 2484,
 2486,
 2488,
 2490,
 2492,
 2494,
 2496,
 2498,
 2500,
 2502,
 2504,
 2506,
 2476,
 2478,
 2480,
 2564,
 2574,
 2856]

In [65]:
image = Image.open(BytesIO(imgdata))

In [66]:
image

In [44]:
images = page.get_images()

In [49]:
page.get_image_info()

[]

In [60]:
elements[1].get("class")

['section', 'abstract']

In [58]:
for element in elements:
    print(f'Name: {element.name}  Class: {element.get("class")}')

Name: div  Class: ['article', 'abstract-view']
Name: div  Class: ['section', 'abstract']


In [54]:
element.class_

In [49]:

# Extract the text from the element
text = element.get_text()

# Print the text
print(text)


AttributeError: 'NoneType' object has no attribute 'get_text'